In [2]:
from astropy.io import fits
import numpy as np
import os
import glob
import pandas as pd
from astropy.coordinates import SkyCoord
import fitsio

In [11]:
# Julius function to read target list 

def read_target_list(filename):

    """
    Function to read a target list from a .csv file and add the SkyCoord objects as a new column. \n
    It further renames the columns "TARGET_RA" and "TARGET_DEC" to "RA" and "dec"
 
    Parameters
    ----------

    filename: str
        filename is the path specifying where to get the target list as a .csv file from

    Returns
    -------

    file: pd.DataFrame
        target list containing the .csv files information and the added SkyCoords as a pandas data frame object
    """

    # Read from a file
    print("Reading csv table...")
    file = pd.read_csv(filename)
    
    # Rename columns
    file = file.rename(columns = {"TARGET_RA": "RA", "TARGET_DEC": "dec"})

    # Calculate and add SkyCoords for AltAz calculation later
    print("Adding SkyCoords to it...")
    sky_coords = SkyCoord(file["RA"], file["dec"], unit = "deg")

    file["SKY_COORD"] = sky_coords
    
    #Return the Pandas dataframe (i.e. The unmodified initial target list)
    print("Finished, now returning pandas df")
    return file

# target_list = read_target_list('../Galaxy Selection/PGAL/PGAL_S230619bg.ecsv')

In [12]:
target_list = read_target_list('../Galaxy Selection/PGAL/PGAL_S230619bg.ecsv')
list_of_all_wst_r_band_images = glob.glob("/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_*[0123456789].fits")
print(list_of_all_wst_r_band_images)

Reading csv table...
Adding SkyCoords to it...
Finished, now returning pandas df
['/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_083.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_072.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_087.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_076.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_093.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_062.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_053.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_097.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624_057.fits', '/project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r_wfi_230624

In [8]:
# So swarp doesnt check every image

def get_list_of_wst_overlapping_images(ra, dec, list_of_wst_images, distance=35):
    
    goodlist = ""
    
    for image in list_of_wst_images:
        #header = fits.read_header(image)
        # check if CRVAL1,2 are less than [distance] arcmin away from [ra,dec]
        # if yes, append to goodlist
        hdul = fits.open(image)
        #header = fitsio.read_header(image) #
        header = hdul[0].header
        crval1 = header["CRVAL1"] # RA
        crval2 = header["CRVAL2"] # Dec
        delt_ra = (ra - crval1) * 60 # [arcmin]
        delt_dec = (dec - crval2) * 60 # [arcmin]
        #print(crval1,ra,delt_ra)
        if np.abs(delt_ra)< distance and np.abs(delt_dec)<distance: 
            goodlist = goodlist + " " + image
    return goodlist

def get_output_filename(ra,dec,prefix=""):
        return prefix+f"galaxy_ra{round(ra,3)}_dec{round(dec,3)}.fits"


#get_list_of_wst_overlapping_images(110.293783220425,63.6778914381844, list_of_all_wst_r_band_images, distance = 35)

In [18]:
# os.sys("module load swarp")

for ra,dec in zip(target_list['RA'],target_list['dec']):
    #list_of_relevant_images = get_list_of_wst_overlapping_images(ra,dec,list_of_all_wst_r_band_images, distance = 35)
    #if(list_of_relevant_images == ""):
    #    print("# ",ra,dec," is not in our footprint")
        
    output_filename = get_output_filename(ra,dec)
    print("swarp -c swarp.conf -IMAGEOUT_NAME=" + output_filename + " -CENTER="+str(ra)+","+str(dec) + " /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r*[0-9].fits") # + list_of_relevant_images)
    output_filename = get_output_filename(ra,dec,"ls")
    print("swarp -c swarp.conf -IMAGEOUT_NAME=" + output_filename + " -CENTER="+str(ra)+","+str(dec) +  " /project/ls-gruen/ligodesi/ls/*r.fits")
    
    #os.sys("swarp -c swarp.conf -CENTER="+str(galaxy['ra'])+","+galaxy['dec'] + " " + list_of_relevant_images) 
    

swarp -c swarp.conf -IMAGEOUT_NAME=galaxy_ra109.68_dec63.186.fits -CENTER=109.679989646446,63.1859641956687 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra109.68_dec63.186.fits -CENTER=109.679989646446,63.1859641956687 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=galaxy_ra110.294_dec63.678.fits -CENTER=110.293783220425,63.6778914381844 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra110.294_dec63.678.fits -CENTER=110.293783220425,63.6778914381844 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=galaxy_ra110.314_dec63.65.fits -CENTER=110.314138321524,63.6504317896136 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_r*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra110.314_dec63.65.fits -CENTER=110.314138321524,63.6504317896136 /project/ls-gruen/ligodes

In [17]:
list_of_all_wst_r_band_images = glob.glob("/project/ls-gruen/ligodesi/ls/*r.fits")

for ra,dec in zip(target_list['RA'],target_list['dec']):
    #list_of_relevant_images = get_list_of_wst_overlapping_images(ra,dec,list_of_all_wst_r_band_images, distance = 35)
    #if(list_of_relevant_images == ""):
    #    print("# ",ra,dec," is not in our footprint")
        
    output_filename = get_output_filename(ra,dec,"ls")
    print("swarp -c swarp.conf -IMAGEOUT_NAME=" + output_filename + " -CENTER="+str(ra)+","+str(dec) +  " /project/ls-gruen/ligodesi/ls/*r.fits")
    #os.sys("swarp -c swarp.conf -CENTER="+str(galaxy['ra'])+","+galaxy['dec'] + " " + list_of_relevant_images) 
    

swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra109.68_dec63.186.fits -CENTER=109.679989646446,63.1859641956687 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra110.294_dec63.678.fits -CENTER=110.293783220425,63.6778914381844 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra110.314_dec63.65.fits -CENTER=110.314138321524,63.6504317896136 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra108.194_dec63.341.fits -CENTER=108.194045155888,63.3413568832094 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra108.122_dec63.391.fits -CENTER=108.121785292373,63.3910844316621 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra109.123_dec63.297.fits -CENTER=109.122728489108,63.2971898203781 /project/ls-gruen/ligodesi/ls/*r.fits
swarp -c swarp.conf -IMAGEOUT_NAME=lsgalaxy_ra108.534_dec63.635.fits -CENTER=108.533591232646,63.63508001621

In [16]:
for galaxy in target_list:
    print(galaxy)

RA
dec
SERSIC
Z
P_GAL
SKY_COORD


In [7]:
list_of_all_ls_r_band_images = glob.glob("/project/ls-gruen/ligodesi/ls/legacy*r.fits")
print(list_of_all_ls_r_band_images)

['/project/ls-gruen/ligodesi/ls/legacysurvey-1050p620-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1053p625-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1049p637-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1054p622-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1048p642-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1056p620-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1054p627-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1052p617-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1054p637-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1050p640-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1050p635-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1053p630-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1048p625-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-1055p640-image-r.fits', '/project/ls-gruen/ligodesi/ls/legacysurvey-104

In [20]:
for ra,dec in zip(target_list['RA'],target_list['dec']):
    #list_of_relevant_images = get_list_of_wst_overlapping_images(ra,dec,list_of_all_ls_r_band_images, distance = 20)
    #if(list_of_relevant_images == ""):
    #    print("# ",ra,dec," is not in our footprint")
    #    continue
        
    output_filename = get_output_filename(ra,dec,"ls")
    print("swarp -c swarp.conf -WEIGHT_TYPE=NONE -IMAGEOUT_NAME=" + output_filename + " -CENTER="+str(ra)+","+str(dec) + " /project/ls-gruen/ligodesi/ls/legacy*r.fits")
    #os.sys("swarp -c swarp.conf -CENTER="+str(galaxy['ra'])+","+galaxy['dec'] + " " + list_of_relevant_images) 
    

swarp -c swarp.conf -WEIGHT_TYPE=NONE -IMAGEOUT_NAME=lsgalaxy_ra109.68_dec63.186.fits -CENTER=109.679989646446,63.1859641956687 /project/ls-gruen/ligodesi/ls/legacy*r.fits
swarp -c swarp.conf -WEIGHT_TYPE=NONE -IMAGEOUT_NAME=lsgalaxy_ra110.294_dec63.678.fits -CENTER=110.293783220425,63.6778914381844 /project/ls-gruen/ligodesi/ls/legacy*r.fits
swarp -c swarp.conf -WEIGHT_TYPE=NONE -IMAGEOUT_NAME=lsgalaxy_ra110.314_dec63.65.fits -CENTER=110.314138321524,63.6504317896136 /project/ls-gruen/ligodesi/ls/legacy*r.fits
swarp -c swarp.conf -WEIGHT_TYPE=NONE -IMAGEOUT_NAME=lsgalaxy_ra108.194_dec63.341.fits -CENTER=108.194045155888,63.3413568832094 /project/ls-gruen/ligodesi/ls/legacy*r.fits
swarp -c swarp.conf -WEIGHT_TYPE=NONE -IMAGEOUT_NAME=lsgalaxy_ra108.122_dec63.391.fits -CENTER=108.121785292373,63.3910844316621 /project/ls-gruen/ligodesi/ls/legacy*r.fits
swarp -c swarp.conf -WEIGHT_TYPE=NONE -IMAGEOUT_NAME=lsgalaxy_ra109.123_dec63.297.fits -CENTER=109.122728489108,63.2971898203781 /project

In [21]:
for ra,dec in zip(target_list['RA'],target_list['dec']):
    #list_of_relevant_images = get_list_of_wst_overlapping_images(ra,dec,list_of_all_wst_r_band_images, distance = 35)
    #if(list_of_relevant_images == ""):
    #    print("# ",ra,dec," is not in our footprint")
        
    output_filename = get_output_filename(ra,dec,"z")
    print("swarp -c swarp.conf -IMAGEOUT_NAME=" + output_filename + " -CENTER="+str(ra)+","+str(dec) + " /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_z*[0-9].fits") # + list_of_relevant_images)
    
for ra,dec in zip(target_list['RA'],target_list['dec']):
    output_filename = get_output_filename(ra,dec,"zls")
    print("swarp -c /project/ls-gruen/ligodesi/ls/swarp.conf -IMAGEOUT_NAME=" + output_filename + " -CENTER="+str(ra)+","+str(dec) +  " /project/ls-gruen/ligodesi/ls/*z.fits")
    
    #os.sys("swarp -c swarp.conf -CENTER="+str(galaxy['ra'])+","+galaxy['dec'] + " " + list_of_relevant_images) 
    

swarp -c swarp.conf -IMAGEOUT_NAME=zgalaxy_ra109.68_dec63.186.fits -CENTER=109.679989646446,63.1859641956687 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_z*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=zgalaxy_ra110.294_dec63.678.fits -CENTER=110.293783220425,63.6778914381844 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_z*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=zgalaxy_ra110.314_dec63.65.fits -CENTER=110.314138321524,63.6504317896136 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_z*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=zgalaxy_ra108.194_dec63.341.fits -CENTER=108.194045155888,63.3413568832094 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_z*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=zgalaxy_ra108.122_dec63.391.fits -CENTER=108.121785292373,63.3910844316621 /project/ls-gruen/ligodesi/V230624-ligodesi_S230619bg_070355_624736_z*[0-9].fits
swarp -c swarp.conf -IMAGEOUT_NAME=zgalaxy_ra10